In [ ]:
import requests
import pandas as pd
import numpy as np

from google.cloud import bigquery

In [ ]:
# Remember naisdevice access to prod-fss
r_team = requests.get(url='https://teamkatalog-api.intern.nav.no/team?status=ACTIVE')
r_po = requests.get(url='https://teamkatalog-api.intern.nav.no/productarea?status=ACTIVE')

In [ ]:
df_team = pd.json_normalize(r_team.json()["content"])
df_po = pd.json_normalize(r_po.json()["content"])

# merge team og po
df_all = df_team.merge(df_po, left_on="productAreaId", right_on="id", suffixes=["_team","_po"])

In [ ]:
client = bigquery.Client("nada-prod-6977")
# hent datasett per naisteam
df_dp = client.query("select REGEXP_EXTRACT(owner, '([^@]+)') naisteam, count(*) dp_count from `nada-prod-6977.dataproducts.dataproducts` where version = (select max(version) from `nada-prod-6977.dataproducts.dataproducts`) group by REGEXP_EXTRACT(owner, '([^@]+)')").result().to_dataframe()

In [ ]:
df_tk = df_all.loc[:,["id_team", "name_team", "name_po", "naisTeams"]]

In [ ]:
# Kobler manuelt naisteams som ikke er registrert på et team i teamkatalogen.
# naisteam: id_team (teamkatalog)
tk_link = {
    "arbeidsforhold": "29d44f3f-ff09-477c-b26a-83cd0aa66116",
    "yrkesskade": "270852c4-4c28-49a8-8189-a16897b31b95",
    "team-ai": "76f378c5-eb35-42db-9f4d-0e8197be0131",
    "spenn": "3c411c72-9587-4c6f-937e-2a404935b778",
    "nada": "5ade590e-3bc4-47fb-8b8d-552392f46376",
    "toi": "0150fd7c-df30-43ee-944e-b152d74c64d6",
    "teamia": "e6e3ce19-2cd3-491b-9ad9-817c2e9b7d96"
          }

In [ ]:
def link_naisteam_to_tk(naisteam):
    unassigned = True
    i = 0
    while i < df_tk.shape[0]:
        if naisteam in df_tk.loc[i, "naisTeams"]:
            return df_tk.loc[i, "id_team"]
        elif naisteam in tk_link:
            return tk_link[naisteam]
        elif i == df_tk.shape[0] - 1:
            return "unknown"
        i += 1

In [ ]:
df_dp["id_team"] = df_dp.naisteam.apply(lambda naisteam: link_naisteam_to_tk(naisteam))

In [ ]:
df_merged = df_tk.merge(df_dp, on="id_team", how="left")

In [ ]:
df_merged = df_merged.sort_values(["name_po", "dp_count"]).reset_index(drop=True)
df_merged["dp_count"] = df_merged.dp_count.fillna(0).astype(int)

In [ ]:
df_merged.head()

In [ ]:
print(df_dp.dp_count.sum(), df_merged.dp_count.sum())

In [ ]:
df_unassigned = df_dp[df_dp["id_team"]=="unknown"]
if df_unassigned.shape[0] > 0:
    df_merged = pd.concat([df_merged, df_unassigned]).fillna(value={"name_team": "unknown", "name_po": "unknown"})

df_merged.tail()

In [ ]:
import plotly.express as px

In [ ]:
fig = px.bar(df_merged, "name_po", "dp_count", color="name_team")
fig.update_layout(height=500)

In [ ]:
# replace dropper tabellen og lager den på nytt. Det betyr at tilgangen må legges til på nytt på markedsplassen
df_merged.to_gbq('dataproducts.po_teams', 
                 'nada-prod-6977',
                 if_exists='replace'
                 )